In [1]:
import os
import sys
import glob
import pandas as pd

from multiprocessing import Pool
import multiprocessing as mp

orig_path = '/pghbio/dbmi/batmanlab/Data/UPMC_Lung_Images/SCCOR/CT_Exams/{subj}/{visitdate}/{subdir}/'
regen_path = '/pylon5/ac5616p/debdas/SCCORProcessing/nifti/{subj}/{visitdate}/{subdir}/'

In [2]:
ord_dir = '/pghbio/dbmi/batmanlab/Data/UPMC_Lung_Images/SCCOR/CT_Exams/'
orig_subjs = os.listdir(ord_dir)
print('total_orig_subjs:',len(orig_subjs))
subj_date_appoint = []
for subj in orig_subjs:
    list_dates = os.listdir(ord_dir+subj+'/')
    for appoint_date in list_dates:
        list_appointment = os.listdir(ord_dir+subj+'/'+appoint_date+'/')
        for appoint in list_appointment:
            subj_date_appoint.append(subj+'|'+appoint_date+'|'+appoint)

total_orig_subjs: 754


In [3]:
len(subj_date_appoint)

5882

In [4]:
orig_phase1_dicom_subjs = [subj for subj in subj_date_appoint if os.path.exists(orig_path.format(subj=subj.split('|')[0],visitdate=subj.split('|')[1],subdir=subj.split('|')[2]))]
print('orig_phase1_dicom_subjs:',len(orig_phase1_dicom_subjs))

orig_phase1_dicom_subjs: 5882


In [5]:
def count_file_size(file_path):
    file_size = os.stat(file_path).st_size
    file_mb_size = int(file_size / 1048576)
    
    return file_mb_size

In [8]:
def count_steps(subj_date_appoint):
    
    subj=subj_date_appoint.split('|')[0]
    visitdate=subj_date_appoint.split('|')[1]
    subdir=subj_date_appoint.split('|')[2]
    
    # We search for the dicom folders for this subject
    orig_dicom_path = orig_path.format(subj=subj,visitdate=visitdate,subdir=subdir)
    orig_dicom_files = os.listdir(orig_dicom_path)
    if len(orig_dicom_files) > 0:
        regen_nifti_path = regen_path.format(subj=subj,visitdate=visitdate,subdir=subdir)
        regen_nifti_file = os.path.join(regen_nifti_path,subdir + '.nii.gz')

        # NIfTI file
        if os.path.exists(regen_nifti_file):
            nifti_size = count_file_size(regen_nifti_file)
            if nifti_size < 1:
                return (nifti_size, regen_nifti_file, orig_dicom_path, len(orig_dicom_files), subdir)

In [18]:
# To boost up the speed, we run counting parallelly
processes = mp.cpu_count()
pool = Pool(processes=processes)

# Calculate a good chunksize (based on implementation of pool.map)
chunksize, extra = divmod(len(orig_phase1_dicom_subjs), 4 * processes)
if extra:
    chunksize += 1

results = []

for result in pool.imap_unordered(count_steps, (subj for subj in orig_phase1_dicom_subjs), chunksize=chunksize):
    if result is not None:
        results = results + [result]
print(len(results))
steps_columns = ['DicomFileCount', 'DicomName', 'NiftiName', 'NiftiFileSize', 'Series']
steps_df = pd.DataFrame(columns = steps_columns)
for DicomFileCount, DicomName, NiftiName, NiftiSize, Series in results:
    row = [DicomFileCount, DicomName, NiftiName, NiftiSize, Series]
    steps_df.loc[steps_df.shape[0]] = row

pool.close()
pool.join()

1390


In [19]:
steps_df.head()

,DicomFileCount,DicomName,NiftiName,NiftiFileSize,Series
0,0,/pylon5/ac5616p/debdas/SCCORProcessing/nifti/1...,/pghbio/dbmi/batmanlab/Data/UPMC_Lung_Images/S...,1,001
1,0,/pylon5/ac5616p/debdas/SCCORProcessing/nifti/1...,/pghbio/dbmi/batmanlab/Data/UPMC_Lung_Images/S...,1,001
2,0,/pylon5/ac5616p/debdas/SCCORProcessing/nifti/1...,/pghbio/dbmi/batmanlab/Data/UPMC_Lung_Images/S...,1,002
3,0,/pylon5/ac5616p/debdas/SCCORProcessing/nifti/1...,/pghbio/dbmi/batmanlab/Data/UPMC_Lung_Images/S...,1,001
4,0,/pylon5/ac5616p/debdas/SCCORProcessing/nifti/1...,/pghbio/dbmi/batmanlab/Data/UPMC_Lung_Images/S...,2,001


In [20]:
len(steps_df)

1390

In [24]:
steps_df.Series.unique()

array(['001', '002', '004', '400'], dtype=object)

In [25]:
steps_df.groupby('Series')['DicomName'].nunique()

Series
001    1367
002      21
004       1
400       1
Name: DicomName, dtype: int64

In [21]:
steps_fn = 'corrupted_file.csv'
steps_df.to_csv(os.path.join(steps_fn),index=False)